# TigerGraph Graph Data Science Library 101 - Community Detection Algorithm

This notebook shows the examples of using the most common community detection algorithms in TigerGraph Graph Science Library. More detailed explanations of these algorithms can be four in the official documentation (https://docs.tigergraph.com/graph-ml/current/community-algorithms/).  

## Step 1: Setting things up
- Connect and Load data
- Visualize the graph schema 
- Get basic stats, e.g., counts of nodes & edges


Connect and Load data: We will be loading the db connection info from our config.json


In [1]:
from pyTigerGraph.datasets import Datasets

dataset = Datasets("ldbc_snb")

Downloading:   0%|          | 0/286678171 [00:00<?, ?it/s]

In [2]:
from pyTigerGraph import TigerGraphConnection
import json

# Read in DB configs
with open('../config.json', "r") as config_file:
    config = json.load(config_file)

conn = TigerGraphConnection(
    host=config["host"],
    username=config["username"],
    password=config["password"],
)

In [3]:
conn.ingestDataset(dataset, getToken=config["getToken"])

---- Checking database ----
A graph with name ldbc_snb already exists in the database. Skip ingestion.


Visualize the graph schema: Adding visualization to would help better understand the algorithm results. In this tutorial, we will be using ipycytoscape to for visualization, and we will be loading the graph schema from TigerGraph Database. 


In [4]:
from pyTigerGraph.visualization import drawSchema

drawSchema(conn.getSchema(force=True))

CytoscapeWidget(cytoscape_layout={'name': 'circle', 'animate': True, 'padding': 1}, cytoscape_style=[{'selecto…

Get stats of the graph data (e.g., number of nodes and edges). 

In [5]:
vertices = conn.getVertexTypes()
for vertex in vertices:
    print("Node count: ({} : {}) ".format(vertex, conn.getVertexCount(vertex)))

Node count: (Comment : 2052169) 
Node count: (Post : 1003605) 
Node count: (Company : 1575) 
Node count: (University : 6380) 
Node count: (City : 1343) 
Node count: (Country : 111) 
Node count: (Continent : 6) 
Node count: (Forum : 90492) 
Node count: (Person : 9892) 
Node count: (Tag : 16080) 
Node count: (Tag_Class : 71) 


In [6]:
import pprint
print("Edges count: ")
pprint.pprint(conn.getEdgeCount())

Edges count: 
{'Container_Of': 1003605,
 'Container_Of_Reverse': 1003605,
 'Has_Creator': 3055774,
 'Has_Creator_Reverse': 3055774,
 'Has_Interest': 229166,
 'Has_Interest_Reverse': 229166,
 'Has_Member': 1611869,
 'Has_Member_Reverse': 1611869,
 'Has_Moderator': 90492,
 'Has_Moderator_Reverse': 90492,
 'Has_Tag': 3721417,
 'Has_Tag_Reverse': 3721417,
 'Has_Type': 16080,
 'Has_Type_Reverse': 16080,
 'Is_Located_In': 3073621,
 'Is_Located_In_Reverse': 3073621,
 'Is_Part_Of': 1454,
 'Is_Part_Of_Reverse': 1454,
 'Is_Subclass_Of': 70,
 'Is_Subclass_Of_Reverse': 70,
 'Knows': 180623,
 'Likes': 2190095,
 'Likes_Reverse': 2190095,
 'Reply_Of': 2052169,
 'Reply_Of_Reverse': 2052169,
 'Study_At': 7949,
 'Study_At_Reverse': 7949,
 'Work_At': 21654,
 'Work_At_Reverse': 21654}


## Step 2: Leveraging pyTigerGraph’s featurizer to run Community Detection algorithms

pyTIgerGraph provides a full suit of data science capabilities, and in this tutorial, we will showcase how to use featurizer to list out all available Centrality algorithms in our GDS library, and to run a few popular algorithms as an example.

In [6]:
feat = conn.gds.featurizer()

In [7]:
feat.listAlgorithms("Community")

Available algorithms for Community:
  connected_components:
    strongly_connected_components:
      01. name: tg_scc
  k_core:
    02. name: tg_kcore
  label_propagation:
    03. name: tg_label_prop
  local_clustering_coefficient:
    04. name: tg_lcc
  louvain:
    05. name: tg_louvain
  triangle_counting:
    06. name: tg_tri_count_fast
Call runAlgorithm() with the algorithm name to execute it


## tg_scc
A strongly connected component (SCC) is a subgraph such that there is a path from any vertex to every other vertex. A graph can contain more than one separate SCC. This SCC algorithm finds the maximal SCCs within a graph. (https://docs.tigergraph.com/graph-ml/current/community-algorithms/strongly-connected-components)

In [8]:
params =  {
    "v_type_set": ["Person"],
    "e_type_set": ["Knows"],
    "reverse_e_type_set": ["Knows"],
    "top_k_dist": 10,
    "print_limit": 10,
    "maximum_iteration": 50,
    "iter_wcc": 5,
    "print_results": False,
    "result_attribute": "tg_scc",
}

In [9]:
import csv
import os
import time
import psutil
!pip install memory_profiler
%load_ext memory_profiler

algo_performance_out = '/home/tigergraph/GraphML/output/algorithm_' + config["job_id"] + '.csv'

start_time = time.time()

algo_memory = %memit -r 1 -o feat.runAlgorithm("tg_scc", params=params)

algo_memory = str(algo_memory)

start = algo_memory.find(": ") + 1
end = algo_memory.find("M")

algo_memory = algo_memory[start:end].strip()

execution_time = time.time() - start_time

cpu_usage = psutil.cpu_percent(4)

print('The CPU usage is: ', cpu_usage)

# print('RAM memory % used:', psutil.virtual_memory()[2])

host_memory = psutil.virtual_memory()[3]/1000000000

print('RAM Used (GB):', host_memory)

print ('tg_scc executed successfully')

print ('execution time: ' + str(execution_time) + ' seconds\n')

algo_id = "tg_scc_" + config["job_id"]

nb_id = "community.ipynb_" + config["job_id"]

keyword = "tg_scc"

data = [algo_id, "false" ,cpu_usage, algo_memory, execution_time, host_memory, "3.8", "no error", nb_id, keyword]

with open(algo_performance_out, mode='a+', encoding='utf-8') as f:
    writer = csv.writer(f) 
    writer.writerow(data)
res = feat.runAlgorithm("tg_scc", params=params)

/opt/conda/lib/python3.9/site-packages/memory_profiler.py:1136: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  ipython_version = LooseVersion(IPython.__version__)
/opt/conda/lib/python3.9/site-packages/setuptools/_distutils/version.py:346: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)


Altering graph schema to save results...
The job add_VERTEX_attr_susmAk completes in 69.328 seconds!
Installing and optimizing the queries, it might take a minute...
Queries installed successfully
peak memory: 124.29 MiB, increment: 1.67 MiB
The CPU usage is:  32.6
RAM Used (GB): 10.552430592
tg_scc executed successfully
execution time: 115.15825653076172 seconds



In [11]:
df = conn.getVertexDataFrame("Person", limit=100_000)
display(df)

,v_id,id,first_name,last_name,gender,birthday,creation_date,location_ip,browser_used,speaks,email,pagerank,pagerank_wt,article_rank,tg_scc
0,32985348841576,32985348841576,Fritz,Becker,female,1983-07-29 00:00:00,2012-07-24 06:25:03,31.13.171.49,Chrome,"[de, en]",[Fritz32985348841576@gmail.com],0.29449,0.15,3.425802e+30,1
1,19791209301554,19791209301554,Carlos,Santos,female,1986-06-28 00:00:00,2011-07-23 00:11:56,198.12.38.156,Firefox,"[pt, en]",[Carlos19791209301554@yahoo.com],1.41297,0.15,2.700411e+31,1
2,15393162790168,15393162790168,Moses,Znaimer,male,1980-08-17 00:00:00,2011-04-11 03:03:01,77.244.155.199,Firefox,"[tg, ru, en]","[Moses15393162790168@gmail.com, Moses153931627...",0.61351,0.15,1.104205e+31,1
3,6597069769941,6597069769941,Manuel,Perez,male,1983-02-09 00:00:00,2010-08-05 01:03:35,31.177.51.194,Internet Explorer,"[es, en]","[Manuel6597069769941@gmail.com, Manuel65970697...",1.02073,0.15,2.007961e+31,1
4,32985348835764,32985348835764,Andrew,Yang,female,1989-01-29 00:00:00,2012-08-03 10:49:17,49.128.82.58,Firefox,"[zh, en]",[Andrew32985348835764@gmail.com],0.19768,0.15,9.344614e+29,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9887,10995116286967,10995116286967,Li,Wang,female,1980-06-22 00:00:00,2010-12-20 02:17:26,27.98.218.4,Firefox,"[zh, en]","[Li10995116286967@gmail.com, Li10995116286967@...",0.21135,0.15,1.489761e+30,1
9888,2199023258090,2199023258090,Abderraouf,David,male,1984-02-06 00:00:00,2010-03-28 15:46:35,192.68.138.187,Internet Explorer,"[ar, fr, en]","[Abderraouf2199023258090@gmail.com, Abderraouf...",0.31669,0.15,3.608502e+30,1
9889,17592186053731,17592186053731,Antonio,Chavez,female,1988-12-09 00:00:00,2011-06-05 14:51:34,187.161.83.235,Chrome,"[es, en]","[Antonio17592186053731@hotmail.com, Antonio175...",1.15352,0.15,2.076123e+31,1
9890,4398046511145,4398046511145,John,Kumar,male,1986-09-22 00:00:00,2010-05-19 01:14:14,27.116.33.147,Safari,"[gu, mr, en]","[John4398046511145@gmail.com, John439804651114...",1.01440,0.15,1.954525e+31,1


In [12]:
df["tg_scc"].value_counts()

1           9163
15728918       1
14680070       1
14680075       1
14680077       1
            ... 
28311731       1
28311742       1
28311758       1
28311770       1
307            1
Name: tg_scc, Length: 730, dtype: int64

## tg_kcore
A k-core of a graph is a maximal connected subgraph in which every vertex is connected to at least 𝑘 vertices in the subgraph. (https://docs.tigergraph.com/graph-ml/current/community-algorithms/k-core-decomposition)

In [13]:
params = {
    "v_type": "Person",
    "e_type": "Knows",
    "k_min": 0,
    "k_max": -1,
    "print_results": False,
    "result_attribute": "tg_kcore",
    "file_path": "",
    "print_all_k": False,
    "show_shells": False
}

In [14]:
start_time = time.time()

algo_memory = %memit -r 1 -o feat.runAlgorithm("tg_kcore", params = params)

algo_memory = str(algo_memory)

start = algo_memory.find(": ") + 1
end = algo_memory.find("M")

algo_memory = algo_memory[start:end].strip()

execution_time = time.time() - start_time

cpu_usage = psutil.cpu_percent(4)

print('The CPU usage is: ', cpu_usage)

# print('RAM memory % used:', psutil.virtual_memory()[2])

host_memory = psutil.virtual_memory()[3]/1000000000

print('RAM Used (GB):', host_memory)

print ('tg_kcore executed successfully')

print ('execution time: ' + str(execution_time) + ' seconds\n')

algo_id = "tg_kcore_" + config["job_id"]

nb_id = "community.ipynb_" + config["job_id"]

keyword = "tg_kcore"

data = [algo_id, "false" ,cpu_usage, algo_memory, execution_time, host_memory, "3.8", "no error", nb_id, keyword]

with open(algo_performance_out, mode='a+', encoding='utf-8') as f:
    writer = csv.writer(f) 
    writer.writerow(data)


Altering graph schema to save results...
The job add_VERTEX_attr_Ebzptv completes in 26.866 seconds!
Installing and optimizing the queries, it might take a minute...
Queries installed successfully
peak memory: 153.48 MiB, increment: 0.09 MiB
The CPU usage is:  31.6
RAM Used (GB): 10.61070848
tg_kcore executed successfully
execution time: 68.03278827667236 seconds



In [15]:
df = conn.getVertexDataFrame("Person", limit=100_000)
display(df)
df["tg_kcore"].value_counts()

,v_id,id,first_name,last_name,gender,birthday,creation_date,location_ip,browser_used,speaks,email,pagerank,pagerank_wt,article_rank,tg_scc,tg_kcore
0,32985348841576,32985348841576,Fritz,Becker,female,1983-07-29 00:00:00,2012-07-24 06:25:03,31.13.171.49,Chrome,"[de, en]",[Fritz32985348841576@gmail.com],0.29449,0.15,3.425802e+30,1,7
1,19791209301554,19791209301554,Carlos,Santos,female,1986-06-28 00:00:00,2011-07-23 00:11:56,198.12.38.156,Firefox,"[pt, en]",[Carlos19791209301554@yahoo.com],1.41297,0.15,2.700411e+31,1,42
2,15393162790168,15393162790168,Moses,Znaimer,male,1980-08-17 00:00:00,2011-04-11 03:03:01,77.244.155.199,Firefox,"[tg, ru, en]","[Moses15393162790168@gmail.com, Moses153931627...",0.61351,0.15,1.104205e+31,1,23
3,6597069769941,6597069769941,Manuel,Perez,male,1983-02-09 00:00:00,2010-08-05 01:03:35,31.177.51.194,Internet Explorer,"[es, en]","[Manuel6597069769941@gmail.com, Manuel65970697...",1.02073,0.15,2.007961e+31,1,39
4,32985348835764,32985348835764,Andrew,Yang,female,1989-01-29 00:00:00,2012-08-03 10:49:17,49.128.82.58,Firefox,"[zh, en]",[Andrew32985348835764@gmail.com],0.19768,0.15,9.344614e+29,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9887,10995116286967,10995116286967,Li,Wang,female,1980-06-22 00:00:00,2010-12-20 02:17:26,27.98.218.4,Firefox,"[zh, en]","[Li10995116286967@gmail.com, Li10995116286967@...",0.21135,0.15,1.489761e+30,1,3
9888,2199023258090,2199023258090,Abderraouf,David,male,1984-02-06 00:00:00,2010-03-28 15:46:35,192.68.138.187,Internet Explorer,"[ar, fr, en]","[Abderraouf2199023258090@gmail.com, Abderraouf...",0.31669,0.15,3.608502e+30,1,8
9889,17592186053731,17592186053731,Antonio,Chavez,female,1988-12-09 00:00:00,2011-06-05 14:51:34,187.161.83.235,Chrome,"[es, en]","[Antonio17592186053731@hotmail.com, Antonio175...",1.15352,0.15,2.076123e+31,1,38
9890,4398046511145,4398046511145,John,Kumar,male,1986-09-22 00:00:00,2010-05-19 01:14:14,27.116.33.147,Safari,"[gu, mr, en]","[John4398046511145@gmail.com, John439804651114...",1.01440,0.15,1.954525e+31,1,40


0     729
43    532
42    444
44    406
1     355
4     353
3     345
41    344
2     324
6     314
40    282
7     273
5     261
39    254
8     252
38    232
9     217
11    205
12    192
14    181
10    178
16    175
37    172
20    166
17    165
13    158
36    152
15    150
19    143
18    138
28    133
22    126
26    124
25    123
31    116
29    114
35    113
21    113
24    112
30    110
34    107
33    107
23    107
27    104
32    101
45     90
Name: tg_kcore, dtype: int64

## tg_louvain
The Louvain Method for community detection partitions the vertices in a graph by approximately maximizing the graph’s modularity score. (https://docs.tigergraph.com/graph-ml/current/community-algorithms/louvain)

In [16]:
params = {
    "v_type_set": ["Person"],
    "e_type_set": ["Knowns"],
    "weight_attribute": "",
    "maximum_iteration": 10,
    "result_attribute": "tg_louvain",
    "file_path": "",
    "print_stats": False
}

In [17]:
start_time = time.time()

algo_memory = %memit -r 1 -o feat.runAlgorithm("tg_louvain", params = params)

algo_memory = str(algo_memory)

start = algo_memory.find(": ") + 1
end = algo_memory.find("M")

algo_memory = algo_memory[start:end].strip()

execution_time = time.time() - start_time

cpu_usage = psutil.cpu_percent(4)

print('The CPU usage is: ', cpu_usage)

# print('RAM memory % used:', psutil.virtual_memory()[2])

host_memory = psutil.virtual_memory()[3]/1000000000

print('RAM Used (GB):', host_memory)

print ('tg_louvain executed successfully')

print ('execution time: ' + str(execution_time) + ' seconds\n')

algo_id = "community_tg_louvain_" + config["job_id"]

nb_id = "community.ipynb_" + config["job_id"]

keyword = "community_tg_louvain"

data = [algo_id, "false" ,cpu_usage, algo_memory, execution_time, host_memory, "3.8", "no error", nb_id, keyword]

with open(algo_performance_out, mode='a+', encoding='utf-8') as f:
    writer = csv.writer(f) 
    writer.writerow(data)

Altering graph schema to save results...
The job add_VERTEX_attr_tLtyph completes in 52.418 seconds!
Installing and optimizing the queries, it might take a minute...
Queries installed successfully
peak memory: 167.86 MiB, increment: 0.00 MiB
The CPU usage is:  35.4
RAM Used (GB): 10.626502656
tg_louvain executed successfully
execution time: 98.339604139328 seconds



In [20]:
df = conn.getVertexDataFrame("Person", limit=100_000)
display(df)
df["tg_louvain"].value_counts()

,v_id,id,first_name,last_name,gender,birthday,creation_date,location_ip,browser_used,speaks,email,pagerank,pagerank_wt,article_rank,tg_scc,tg_kcore,tg_louvain
0,32985348841576,32985348841576,Fritz,Becker,female,1983-07-29 00:00:00,2012-07-24 06:25:03,31.13.171.49,Chrome,"[de, en]",[Fritz32985348841576@gmail.com],0.29449,0.15,3.425802e+30,1,7,63963136
1,19791209301554,19791209301554,Carlos,Santos,female,1986-06-28 00:00:00,2011-07-23 00:11:56,198.12.38.156,Firefox,"[pt, en]",[Carlos19791209301554@yahoo.com],1.41297,0.15,2.700411e+31,1,42,63963137
2,15393162790168,15393162790168,Moses,Znaimer,male,1980-08-17 00:00:00,2011-04-11 03:03:01,77.244.155.199,Firefox,"[tg, ru, en]","[Moses15393162790168@gmail.com, Moses153931627...",0.61351,0.15,1.104205e+31,1,23,63963138
3,6597069769941,6597069769941,Manuel,Perez,male,1983-02-09 00:00:00,2010-08-05 01:03:35,31.177.51.194,Internet Explorer,"[es, en]","[Manuel6597069769941@gmail.com, Manuel65970697...",1.02073,0.15,2.007961e+31,1,39,63963139
4,32985348835764,32985348835764,Andrew,Yang,female,1989-01-29 00:00:00,2012-08-03 10:49:17,49.128.82.58,Firefox,"[zh, en]",[Andrew32985348835764@gmail.com],0.19768,0.15,9.344614e+29,1,2,63963140
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9887,10995116286967,10995116286967,Li,Wang,female,1980-06-22 00:00:00,2010-12-20 02:17:26,27.98.218.4,Firefox,"[zh, en]","[Li10995116286967@gmail.com, Li10995116286967@...",0.21135,0.15,1.489761e+30,1,3,316
9888,2199023258090,2199023258090,Abderraouf,David,male,1984-02-06 00:00:00,2010-03-28 15:46:35,192.68.138.187,Internet Explorer,"[ar, fr, en]","[Abderraouf2199023258090@gmail.com, Abderraouf...",0.31669,0.15,3.608502e+30,1,8,317
9889,17592186053731,17592186053731,Antonio,Chavez,female,1988-12-09 00:00:00,2011-06-05 14:51:34,187.161.83.235,Chrome,"[es, en]","[Antonio17592186053731@hotmail.com, Antonio175...",1.15352,0.15,2.076123e+31,1,38,318
9890,4398046511145,4398046511145,John,Kumar,male,1986-09-22 00:00:00,2010-05-19 01:14:14,27.116.33.147,Safari,"[gu, mr, en]","[John4398046511145@gmail.com, John439804651114...",1.01440,0.15,1.954525e+31,1,40,319


63963136    1
14680185    1
14680178    1
14680179    1
14680180    1
           ..
28311685    1
28311686    1
28311687    1
28311688    1
320         1
Name: tg_louvain, Length: 9892, dtype: int64

## tg_lcc

The local clustering coefficient of a vertex in a graph quantifies how close its neighbors are to being a complete graph. In a complete graph, every two distinct vertices are connected. (https://docs.tigergraph.com/graph-ml/current/community-algorithms/local-clustering-coefficient)

In [21]:
params = {
    "v_type": "Person",
    "e_type": "Knows",
    "top_k": 100,
    "print_results": False,
    "result_attribute": "tg_lcc",
    "file_path": "",
    "display_edges": False
}

In [22]:
start_time = time.time()

algo_memory = %memit -r 1 -o feat.runAlgorithm("tg_lcc", params = params)

algo_memory = str(algo_memory)

start = algo_memory.find(": ") + 1
end = algo_memory.find("M")

algo_memory = algo_memory[start:end].strip()

execution_time = time.time() - start_time

cpu_usage = psutil.cpu_percent(4)

print('The CPU usage is: ', cpu_usage)

# print('RAM memory % used:', psutil.virtual_memory()[2])

host_memory = psutil.virtual_memory()[3]/1000000000

print('RAM Used (GB):', host_memory)

print ('tg_lcc executed successfully')

print ('execution time: ' + str(execution_time) + ' seconds\n')

algo_id = "tg_lcc_" + config["job_id"]

nb_id = "community.ipynb_" + config["job_id"]

keyword = "tg_lcc"

data = [algo_id, "false" ,cpu_usage, algo_memory, execution_time, host_memory, "3.8", "no error", nb_id, keyword]

with open(algo_performance_out, mode='a+', encoding='utf-8') as f:
    writer = csv.writer(f) 
    writer.writerow(data)

Altering graph schema to save results...
The job add_VERTEX_attr_kOLKNO completes in 42.961 seconds!
Installing and optimizing the queries, it might take a minute...
Queries installed successfully
peak memory: 170.28 MiB, increment: 0.00 MiB
The CPU usage is:  32.3
RAM Used (GB): 10.631847936
tg_lcc executed successfully
execution time: 84.18365383148193 seconds



In [23]:
df = conn.getVertexDataFrame("Person", limit=100_000)
display(df)
df["tg_lcc"].value_counts()

,v_id,id,first_name,last_name,gender,birthday,creation_date,location_ip,browser_used,speaks,email,pagerank,pagerank_wt,article_rank,tg_scc,tg_kcore,tg_louvain,tg_lcc
0,32985348841576,32985348841576,Fritz,Becker,female,1983-07-29 00:00:00,2012-07-24 06:25:03,31.13.171.49,Chrome,"[de, en]",[Fritz32985348841576@gmail.com],0.29449,0.15,3.425802e+30,1,7,63963136,0.19048
1,19791209301554,19791209301554,Carlos,Santos,female,1986-06-28 00:00:00,2011-07-23 00:11:56,198.12.38.156,Firefox,"[pt, en]",[Carlos19791209301554@yahoo.com],1.41297,0.15,2.700411e+31,1,42,63963137,0.11211
2,15393162790168,15393162790168,Moses,Znaimer,male,1980-08-17 00:00:00,2011-04-11 03:03:01,77.244.155.199,Firefox,"[tg, ru, en]","[Moses15393162790168@gmail.com, Moses153931627...",0.61351,0.15,1.104205e+31,1,23,63963138,0.20949
3,6597069769941,6597069769941,Manuel,Perez,male,1983-02-09 00:00:00,2010-08-05 01:03:35,31.177.51.194,Internet Explorer,"[es, en]","[Manuel6597069769941@gmail.com, Manuel65970697...",1.02073,0.15,2.007961e+31,1,39,63963139,0.13953
4,32985348835764,32985348835764,Andrew,Yang,female,1989-01-29 00:00:00,2012-08-03 10:49:17,49.128.82.58,Firefox,"[zh, en]",[Andrew32985348835764@gmail.com],0.19768,0.15,9.344614e+29,1,2,63963140,1.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9887,10995116286967,10995116286967,Li,Wang,female,1980-06-22 00:00:00,2010-12-20 02:17:26,27.98.218.4,Firefox,"[zh, en]","[Li10995116286967@gmail.com, Li10995116286967@...",0.21135,0.15,1.489761e+30,1,3,316,0.33333
9888,2199023258090,2199023258090,Abderraouf,David,male,1984-02-06 00:00:00,2010-03-28 15:46:35,192.68.138.187,Internet Explorer,"[ar, fr, en]","[Abderraouf2199023258090@gmail.com, Abderraouf...",0.31669,0.15,3.608502e+30,1,8,317,0.10714
9889,17592186053731,17592186053731,Antonio,Chavez,female,1988-12-09 00:00:00,2011-06-05 14:51:34,187.161.83.235,Chrome,"[es, en]","[Antonio17592186053731@hotmail.com, Antonio175...",1.15352,0.15,2.076123e+31,1,38,318,0.14711
9890,4398046511145,4398046511145,John,Kumar,male,1986-09-22 00:00:00,2010-05-19 01:14:14,27.116.33.147,Safari,"[gu, mr, en]","[John4398046511145@gmail.com, John439804651114...",1.01440,0.15,1.954525e+31,1,40,319,0.13511


0.00000    1570
0.33333     319
0.16667     229
0.20000     222
0.14286     137
           ... 
0.34615       1
0.04385       1
0.08584       1
0.05530       1
0.12706       1
Name: tg_lcc, Length: 3152, dtype: int64